# Preparation

In [13]:
#spa or eng
language = 'spa'

In [14]:
spanish_words = ["abogado","aceite","áfrica","agente","agua","águila","aguja","agujero","aire","alemania","algodón","alianza","alpes","ambulancia","américa","ángel","anillo","antártida","antorcha","araña","archivo","arco","argentina","artículo","as","atlántida","azteca","baile","bala","ballena","banco","banda","baño","barco","barra","batería","berlín","bermudas","bicho","blanco","bloque","boca","bola","bolsa","bomba","bosque","bota","botella","botón","brazo","bruja","caballero","caballo","cabeza","cabina","cabo","cactus","cadena","caja","cama","cámara","cambio","campana","campo","canal","canguro","canto","caña","capa","capital","caqui","cara","caravana","carga","carrera","carro","carta","casco","casino","caza","cementerio","centauro","centro","cervantes","checo","chocolate","choque","chuleta","científico","cinta","cinturón","círculo","clase","coche","cocinero","coco","código","cola","cólera","columna","cometa","compás","concierto","conejo","contrabandista","copa","corazón","corneta","corona","corredor","corriente","corte","cresta","cromo","cruz","cuadro","cuarto","cubierta","cubo","cuchillo","cuello","cuerda","cuerno","cura","dama","delta","destino","día","diamante","diana","diario","diente","dinosaurio","disco","don","dragón","duende","egipto","embajada","emperador","enano","enfermedad","enfermera","enlace","escorpión","espacio","espía","estación","estadio","estado","estrella","estudio","etiqueta","europa","extraterrestre","falda","fantasma","faro","ficha","fiesta","figura","flauta","flecha","foso","francia","frente","fuego","fuente","fuerza","furgoneta","gancho","gato","genio","gigante","golfo","golondrina","golpe","goma","góndola","gota","grado","granada","grano","grecia","grifo","guante","guardia","guerra","gusano","helado","helicóptero","hielo","hierba","hoja","hollywood","horca","hospital","hotel","iglesia","imán","india","índice","inglaterra","italia","jarra","judía","juicio","kiwi","ladrón","lago ness","láser","látigo","lengua","león","libra","lima","limusina","línea","lista","llama","llave","lomo","londres","luna","luz","maestro","magia","malta","mancha","mando","manga","mango","mano","manzana","mañana","marca","marcha","marfil","masa","máscara","mazo","médico","mercurio","mesa","metro","méxico","micro","microscopi","mielo","millonario","mina","misil","modelo","módulo","monitor","mono","mortero","moscú","motor","muelle","muerte","muñeca","muro","naranja","nave","nieve","nilo","ninja","noche","nota","nudo","nueva york","obra","ojo","ola","olimpo","ópera","orden","órgano","ornitorrinco","oro","oso","pala","palma","pantalla","papel","paracaídas","pase","paso","pasta","pastel","pavo","pekín","película","pelotón","pendiente","perro","pez","pico","pie","pieza","pila","piloto","pincho","pingüino","pinta","piña","pirámide","pirata","pista","pistola","placa","plano","planta","plátano","playa","plomo","pluma","policía","polo","portada","portero","potro","prensa","prima","princesa","puente","puerto","pulpo","pulso","punta","punto","radio","rascacielos","ratón","rayo","red","regla","reina","reserva","revolución","rey","robot","rojo","roma","ronda","rosa","ruleta","sable","sáhara","salsa","satélite","saturno","señal","serie","serpiente","sierra","silla","sirena","sobre","soldado","submarinista","suerte","superhéroe","tabla","tableta","taco","tacto","talón","tanque","tapa","tarde","teatro","teclado","telescopio","testigo","tiempo","tienda","tierra","tokio","topo","torre","trama","tronco","tubería","tubo","unicornio","vacío","vado","vampiro","vela","veneno","venus","vestido","vida","vidrio","viento","yema","zanahoria","zapato"]

In [15]:
english_words = ["AFRICA","AGENT","AIR","ALIEN","ALPS","AMAZON","AMBULANCE","AMERICA","ANGEL","ANTARCTICA","APPLE","ARM","ATLANTIS","AUSTRALIA","AZTEC","BACK","BALL","BAND","BANK","BAR","BARK","BAT","BATTERY","BEACH","BEAR","BEAT","BED","BEIJING","BELL","BELT","BERLIN","BERMUDA","BERRY","BILL","BLOCK","BOARD","BOLT","BOMB","BOND","BOOM","BOOT","BOTTLE","BOW","BOX","BRIDGE","BRUSH","BUCK","BUFFALO","BUG","BUGLE","BUTTON","CALF","CANADA","CAP","CAPITAL","CAR","CARD","CARROT","CASINO","CAST","CAT","CELL","CENTAUR","CENTER","CHAIR","CHANGE","CHARGE","CHECK","CHEST","CHICK","CHINA","CHOCOLATE","CHURCH","CIRCLE","CLIFF","CLOAK","CLUB","CODE","COLD","COMIC","COMPOUND","CONCERT","CONDUCTOR","CONTRACT","COOK","COPPER","COTTON","COURT","COVER","CRANE","CRASH","CRICKET","CROSS","CROWN","CYCLE","CZECH","DANCE","DATE","DAY","DEATH","DECK","DEGREE","DIAMOND","DICE","DINOSAUR","DISEASE","DOCTOR","DOG","DRAFT","DRAGON","DRESS","DRILL","DROP","DUCK","DWARF","EAGLE","EGYPT","EMBASSY","ENGINE","ENGLAND","EUROPE","EYE","FACE","FAIR","FALL","FAN","FENCE","FIELD","FIGHTER","FIGURE","FILE","FILM","FIRE","FISH","FLUTE","FLY","FOOT","FORCE","FOREST","FORK","FRANCE","GAME","GAS","GENIUS","GERMANY","GHOST","GIANT","GLASS","GLOVE","GOLD","GRACE","GRASS","GREECE","GREEN","GROUND","HAM","HAND","HAWK","HEAD","HEART","HELICOPTER","HIMALAYAS","HOLE","HOLLYWOOD","HONEY","HOOD","HOOK","HORN","HORSE","HORSESHOE","HOSPITAL","HOTEL","ICE","ICE CREAM","INDIA","IRON","IVORY","JACK","JAM","JET","JUPITER","KANGAROO","KETCHUP","KEY","KID","KING","KIWI","KNIFE","KNIGHT","LAB","LAP","LASER","LAWYER","LEAD","LEMON","LEPRECHAUN","LIFE","LIGHT","LIMOUSINE","LINE","LINK","LION","LITTER","LOCH NESS","LOCK","LOG","LONDON","LUCK","MAIL","MAMMOTH","MAPLE","MARBLE","MARCH","MASS","MATCH","MERCURY","MEXICO","MICROSCOPE","MILLIONAIRE","MINE","MINT","MISSILE","MODEL","MOLE","MOON","MOSCOW","MOUNT","MOUSE","MOUTH","MUG","NAIL","NEEDLE","NET","NEW YORK","NIGHT","NINJA","NOTE","NOVEL","NURSE","NUT","OCTOPUS","OIL","OLIVE","OLYMPUS","OPERA","ORANGE","ORGAN","PALM","PAN","PANTS","PAPER","PARACHUTE","PARK","PART","PASS","PASTE","PENGUIN","PHOENIX","PIANO","PIE","PILOT","PIN","PIPE","PIRATE","PISTOL","PIT","PITCH","PLANE","PLASTIC","PLATE","PLATYPUS","PLAY","PLOT","POINT","POISON","POLE","POLICE","POOL","PORT","POST","POUND","PRESS","PRINCESS","PUMPKIN","PUPIL","PYRAMID","QUEEN","RABBIT","RACKET","RAY","REVOLUTION","RING","ROBIN","ROBOT","ROCK","ROME","ROOT","ROSE","ROULETTE","ROUND","ROW","RULER","SATELLITE","SATURN","SCALE","SCHOOL","SCIENTIST","SCORPION","SCREEN","SCUBA DIVER","SEAL","SERVER","SHADOW","SHAKESPEARE","SHARK","SHIP","SHOE","SHOP","SHOT","SINK","SKYSCRAPER","SLIP","SLUG","SMUGGLER","SNOW","SNOWMAN","SOCK","SOLDIER","SOUL","SOUND","SPACE","SPELL","SPIDER","SPIKE","SPINE","SPOT","SPRING","SPY","SQUARE","STADIUM","STAFF","STAR","STATE","STICK","STOCK","STRAW","STREAM","STRIKE","STRING","SUB","SUIT","SUPERHERO","SWING","SWITCH","TABLE","TABLET","TAG","TAIL","TAP","TEACHER","TELESCOPE","TEMPLE","THEATER","THIEF","THUMB","TICK","TIE","TIME","TOKYO","TOOTH","TORCH","TOWER","TRACK","TRAIN","TRIANGLE","TRIP","TRUNK","TUBE","TURKEY","UNDERTAKER","UNICORN","VACUUM","VAN","VET","WAKE","WALL","WAR","WASHER","WASHINGTON","WATCH","WATER","WAVE","WEB","WELL","WHALE","WHIP","WIND","WITCH","WORM","YARD"]

In [82]:
import fasttext
import fasttext.util
import gensim
import nltk
import itertools
import pandas as pd
import numpy as np
from gensim.models import FastText, Word2Vec
from gensim.models.fasttext import load_facebook_model
from nltk.corpus import wordnet as wn
from sklearn.cluster import KMeans
import warnings

In [17]:
%%capture
nltk.download("omw-1.4")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\josem\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\josem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
if language == "spa":
    fasttext.util.download_model('es', if_exists='ignore')  # English
else:
    fasttext.util.download_model('en', if_exists='ignore')

In [19]:
%%capture
FAST_TEXT_MODEL = "cc.es.300.bin" # Model name in fasttext


ft = fasttext.load_model(FAST_TEXT_MODEL)

In [20]:
model = load_facebook_model(FAST_TEXT_MODEL)
#model.wv.most_similar(positive=['red', 'green'])

In [21]:
if language == "spa":
    words = spanish_words 
else: 
    words = english_words

# Medias entre palabras

In [22]:
def get_cosine_similarity(model, word1, word2):
    return model.wv.similarity(word1, word2)

def get_synonyms(word1, lang = 'spa'):
    all_synonyms = list(map(lambda x: x.lemma_names(lang), wn.synsets(word1, lang=lang)))
    return list(set(itertools.chain(*all_synonyms)))

def get_antonyms(word1, lang = 'spa'):
    all_lemas = list(map(lambda x: x.lemmas(), wn.synsets(word1, lang=lang)))
    all_antonyms_lemas = list(map(lambda x: x.antonyms(), list(itertools.chain(*all_lemas))))
    all_antonyms = map(lambda x: x.synset().lemma_names(lang=lang), list(itertools.chain(*all_antonyms_lemas)))
    return list(set(itertools.chain(*all_antonyms)))

def get_synonyms_with_similarity(word1, model, lang='spa'):
    return list(map(lambda word2: (word2, get_cosine_similarity(model, word1, word2)), get_synonyms(word1, lang=lang)))

def get_max_synonym_similarity_between_words(word1, word2, model, lang='spa'):
    synonyms_word1 = get_synonyms(word1, lang = lang)
    synonyms_word2 = get_synonyms(word2, lang = lang)
    synonyms = list(set(synonyms_word1).intersection(synonyms_word2))
    if len(synonyms) !=0:
        max_value = max(synonyms)
    else:
        max_value = 0
    max_value = max_value if max_value else 0
    return max_value

def get_number_of_same_synonyms(word1, word2, lang = 'spa'):
    synonyms_word1 = get_synonyms(word1, lang = lang)
    synonyms_word2 = get_synonyms(word2, lang = lang)
    return len(set(synonyms_word1).intersection(synonyms_word2))

def get_number_of_same_antonyms(word1, word2, lang = 'spa'):
    antonyms_word1 = get_antonyms(word1, lang = lang)
    antonyms_word2 = get_antonyms(word2, lang = lang)
    return len(set(antonyms_word1).intersection(antonyms_word2))

def get_number_synonyms_vs_antonyms(word1, word2, lang = 'spa'):
    antonyms_word1 = get_antonyms(word1, lang = lang)
    synonyms_word1 = get_synonyms(word1, lang = lang)
    antonyms_word2 = get_antonyms(word2, lang = lang)
    synonyms_word2 = get_synonyms(word2, lang = lang)
    return len(set(synonyms_word1).intersection(antonyms_word2)) + len(set(antonyms_word1).intersection(synonyms_word2))

def get_antonyms_with_similarity(word1, model, lang='spa'):
    return list(map(lambda word2: (word2, get_cosine_similarity(model, word1, word2)), get_antonyms(word1, lang=lang)))

def get_max_antonym_similarity_between_words(word1, word2, model, lang='spa'):
    antonyms_word1 = get_antonyms(word1, lang = lang)
    antonyms_word2 = get_antonyms(word2, lang = lang)
    antonyms = list(set(antonyms_word1).intersection(antonyms_word2))
    if len(antonyms) != 0:
        max_value = max(antonyms)
    else:
        max_value = 0
    max_value = max_value if max_value else 0
    return max_value

def get_hypernyms_of_synsets(synsets):
    hypernyms = list(map(lambda x: x.hypernyms(), synsets))
    return list(set(itertools.chain(*hypernyms)))

def get_path_similarity(synset1, synset2):
    similarity = wn.path_similarity(synset1, synset2)
    similarity = similarity if similarity else 0
    return similarity

def get_first_common_hypernym(word1, word2, lang = 'spa'):
    temporal_hypernyms1 = get_hypernyms_of_synsets(wn.synsets(word1, lang=lang))
    temporal_hypernyms2 = get_hypernyms_of_synsets(wn.synsets(word2, lang=lang))
    hypernyms1 = list(set(temporal_hypernyms1))
    hypernyms2 = list(set(temporal_hypernyms2))
    while len(set(hypernyms1).intersection(hypernyms2))==0 and (len(temporal_hypernyms1)!=0 or len(temporal_hypernyms2)!=0):
        hypernyms1 = list(set(hypernyms1 + temporal_hypernyms1))
        hypernyms2 = list(set(hypernyms2 + temporal_hypernyms2))
        temporal_hypernyms1 = get_hypernyms_of_synsets(temporal_hypernyms1)
        temporal_hypernyms2 = get_hypernyms_of_synsets(temporal_hypernyms2)
    if len(set(hypernyms1).intersection(hypernyms2))!=0:
        hypernyms = list(set(hypernyms1).intersection(hypernyms2))
        possible_hypernyms = list(map(lambda x: max(list(map(lambda y: get_path_similarity(x, y), wn.synsets(word1, lang=lang)))) + max(list(map(lambda y: get_path_similarity(x, y), wn.synsets(word2, lang=lang)))),hypernyms))
        max_value = max(possible_hypernyms)
        hypernym = hypernyms[possible_hypernyms.index(max_value)]
        similarity_to_word1 = max(list(map(lambda x: get_path_similarity(hypernyms[possible_hypernyms.index(max_value)],x),wn.synsets(word1, lang=lang))))
        similarity_to_word2 = max(list(map(lambda x: get_path_similarity(hypernyms[possible_hypernyms.index(max_value)],x),wn.synsets(word2, lang=lang))))
    else:
        hypernym = None
        similarity_to_word1 = 0
        similarity_to_word2 = 0
    return hypernym, similarity_to_word1, similarity_to_word2
    
    
def apply_all_metrics_to_words_df(df, model, word2, lang = 'spa'):
    df['similarity'] = df.apply(lambda x: get_cosine_similarity(model, x['words'], word2), axis=1)
    df['number_same_synonyms'] = df.apply(lambda x: get_number_of_same_synonyms(x['words'], word2, lang= lang), axis=1)
    df['number_same_antonyms'] = df.apply(lambda x: get_number_of_same_antonyms(x['words'], word2, lang= lang), axis=1)
    df['synonyms_vs_antonyms'] = df.apply(lambda x: get_number_synonyms_vs_antonyms(x['words'], word2, lang= lang), axis=1)
    df['max_similarity_synonym'] = df.apply(lambda x: get_max_synonym_similarity_between_words(x['words'], word2, model, lang=lang), axis=1)
    df['max_similarity_antonym'] = df.apply(lambda x: get_max_antonym_similarity_between_words(x['words'], word2, model, lang=lang), axis=1)
    df['distance_common_hypernym'] =  df.apply(lambda x: get_first_common_hypernym(x['words'], word2, lang= lang)[1], axis=1)
    df['distance_common_hypernym_from_word'] =  df.apply(lambda x: get_first_common_hypernym(x['words'], word2, lang= lang)[2], axis=1)
    return df

In [23]:
#Grado similaridad
#Numero de sinonimos en común
#Número de antónimos en común
#Relación con los sinonimos
#Relación con los antónimos
#Familia semántica (Hiperonimos)
#Número de palabras entre la pista y la evaluada(s)

In [24]:
data = pd.DataFrame(words, columns = ["words"])

In [25]:
sample_data = data.sample(25)

In [26]:
df = apply_all_metrics_to_words_df(sample_data, model, 'dinero', lang=language)
df.index = df.words
df = df.drop(['words'], axis = 1)

# Vectores como variables

In [27]:
def get_dataframe_with_coordinates_vector(words, clue):
    column_names = list(map(lambda x: 'var'+str(x),np.arange(0, 300)))
    df_with_vectors = pd.DataFrame(words, columns = ["words"], index=words)
    df_with_vectors[column_names] = df_with_vectors.apply(lambda x: list(ft.get_word_vector(x['words'])), axis=1, result_type='expand')
    df_with_vectors = df_with_vectors.drop(['words'], axis=1)
    df_with_vectors_and_clue = pd.concat([df_with_vectors, pd.DataFrame([list(ft.get_word_vector(clue))], index=[clue], columns=column_names)])
    return df_with_vectors_and_clue

# Clustering

In [28]:
sample_words = np.random.choice(words, size=25, replace=False)
print(sample_words)

['cometa' 'ladrón' 'cámara' 'banco' 'alianza' 'pila' 'estadio' 'clase'
 'barco' 'vacío' 'barra' 'misil' 'naranja' 'aceite' 'furgoneta' 'pavo'
 'ninja' 'pekín' 'pastel' 'muerte' 'chuleta' 'pie' 'nudo' 'blanco' 'señal']


In [100]:
def get_distance(v1, v2):
    return np.sqrt(np.sum(np.subtract(v1, v2)**2))

def get_words_from_clue(sample_words, clue, number_words):
    df_for_clustering = get_dataframe_with_coordinates_vector(sample_words, clue)
    kmeans = KMeans(n_clusters = 25 - number_words)
    kmeans_response = kmeans.fit(df_for_clustering)
    df_for_clustering['cluster'] = kmeans_response.labels_
    clue_cluster = kmeans_response.cluster_centers_[df_for_clustering['cluster'][-1]]
    distance_to_clue_cluster = list(map(lambda center: get_distance(center, clue_cluster), kmeans_response.cluster_centers_))
    df_for_clustering['distance_to_clue_cluster'] = df_for_clustering.apply(lambda cluster: distance_to_clue_cluster[int(cluster['cluster'])], axis=1)
    return df_for_clustering[df_for_clustering['cluster']==df_for_clustering['cluster'][-1]]['cluster']

In [107]:
warnings.filterwarnings('ignore')
clue='atraco'
number_words = 4
get_words_from_clue(sample_words, clue, number_words)

ladrón    14
atraco    14
Name: cluster, dtype: int32

In [112]:
df_for_clustering = get_dataframe_with_coordinates_vector(sample_words, clue)
kmeans = KMeans(n_clusters = 25 - number_words)
kmeans_response = kmeans.fit(df_for_clustering)
df_for_clustering['cluster'] = kmeans_response.labels_
clue_cluster = kmeans_response.cluster_centers_[df_for_clustering['cluster'][-1]]
distance_to_clue_cluster = list(map(lambda center: get_distance(center, clue_cluster), kmeans_response.cluster_centers_))
df_for_clustering['distance_to_clue_cluster'] = df_for_clustering.apply(lambda cluster: distance_to_clue_cluster[int(cluster['cluster'])], axis=1)

In [113]:
df_for_clustering[['cluster', 'distance_to_clue_cluster']]

,cluster,distance_to_clue_cluster
cometa,14,1.155079
ladrón,7,0.000000
cámara,17,1.080697
banco,10,1.023968
alianza,1,1.078753
pila,3,1.606659
estadio,19,1.092083
clase,16,1.188108
barco,13,1.213781
vacío,8,1.283173
